In [217]:
import numpy as np
import sys
import io
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
import re
import datetime

lowest_prob_accepted = 0.2
def no_accent_vietnamese(s):
    s = re.sub(u'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(u'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(u'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(u'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(u'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(u'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(u'[ìíịỉĩ]', 'i', s)
    s = re.sub(u'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(u'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(u'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(u'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(u'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(u'[Đ]', 'D', s)
    s = re.sub(u'[đ]', 'd', s)
    return s

def get_all_substrings(input_string_array):
    length = len(input_string_array)
    return [input_string[i:j + 1] for i in range(length) for j in range(i, length)]

def author_traindata():
    authorFilepath = './traindata/TacGia.txt'  
    authorF = io.open(authorFilepath, mode="r", encoding="utf-8")
    authors = []
    for i, line in enumerate(authorF):
        data = line.strip().replace(u'\ufeff', '')
        authors.append(data)
        data1 = 'Tác giả ' + data
        authors.append(data1)
        data = no_accent_vietnamese(data)
        authors.append(data)
    return authors

def docName_traindata():
    docFilepath = './traindata/TenTL.txt'  
    docF = io.open(docFilepath, mode="r", encoding="utf-8")
    docNames = []
    for i, line in enumerate(docF):
        data = line.strip().replace(u'\ufeff', '')
        docNames.append(data)
        data1 = 'Giáo trình ' + data
        docNames.append(data1)
        data2 = 'Lý thuyết ' + data
        docNames.append(data2)
        data3 = 'Bài tập ' + data
        docNames.append(data3)
        data = no_accent_vietnamese(data)
        docNames.append(data)
    return docNames

def publisher_traindata():
    publisherFilepath = './traindata/NXB.txt'  
    publisherF = io.open(publisherFilepath, mode="r", encoding="utf-8")
    publishers = []
    for i, line in enumerate(publisherF):
        data = line.strip().replace(u'\ufeff', '')
        publishers.append(data)
        data3 = 'Đại học ' + data
        publishers.append(data3)  
        data = no_accent_vietnamese(data)
        publishers.append(data)  
    return publishers

def another(i):
    if i == 0:
        return 1
    if i == 1:
        return 0
    if i == 2:
        return 0
    return -1

def the_other(i):
    if i == 0:
        return 2
    if i == 1:
        return 2
    if i == 2:
        return 1
    return -1

def get_classified_words(classifier, all_possible_strings, which):
    i = another(which)
    j = the_other(which)
    firstTime = True
    max_vote = 0
    highest_voted_string = ''
    for string in all_possible_strings:
        if len(string) > 1:
            string = ' '.join(string)
            X_test = np.array([string]) 
            vote = classifier.decision_function(X_test)
            if firstTime:
                firstTime = False
                max_vote = vote[0][which]
                highest_voted_string = string
            else:
                if vote[0][which] > max_vote: 
                    max_vote = vote[0][which]
                    highest_voted_string = string
    if max_vote < lowest_prob_accepted:
        return ''
    return highest_voted_string


authors = author_traindata()
docNames = docName_traindata()
publishers = publisher_traindata()

X_train_pre = authors + docNames + publishers
X_train = np.array(X_train_pre)

y_train = []
for x in authors:
    y_train.append(0)
for x in docNames:
    y_train.append(1)
for x in publishers:
    y_train.append(2)

classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC(C=10))])
classifier.fit(X_train, y_train)

#input_string = 'Nguyễn Thị Cẩm Uyên Quà tặng cuộc sống 2 Bách Khoa Hà Nội 2017'
#input_string = 'Bài tập Xác suất thống kê 9 Nguyễn Thanh Loan NXB Đà Nẵng 2001'
#input_string = 'Nguyễn Đức Nghĩa Cấu trúc dữ liệu và thuật toán Bách Khoa Hà Nội 2013'
#input_string = 'NXB Đà Nẵng 2009 Bài tập Tiếng Anh 10 Hà Thanh Uyên'
#input_string = '2016 Tâm lý học trong nháy mắt Phạm Huy Hoàng Thời đại'
#input_string = 'Mai Lan Hương Bài tập trắc nghiệm Tiếng Anh 10 Đà Nẵng'
#input_string = 'Mai Lan Hương 2008 Giải tích 1'
input_string = 'Hóa học đại cương Lương Duyên Bình Bách Khoa Hà Nội 2003'
print('Đầu vào:', input_string)

# Input Preprocessing
input_string = input_string.replace('NXB', '')
input_string = input_string.replace('Nhà xuất bản', '')
input_string = input_string.replace('nhà xuất bản', '')

match = re.search('(?:(?<!\d)\d{4}(?!\d))', input_string)
pubYear = 0
if match != None:
    pubYear = match.group(0)
    input_string = input_string.replace(pubYear, '')
    pubYear = int(pubYear)
    now = datetime.datetime.now()
    if (pubYear < 1700 or pubYear > now.year):
        pubYear = 0
match = re.search(' \d\d', input_string)
docNumber = 0
if match != None:
    docNumber = match.group(0)
    input_string = input_string.replace(docNumber, '')
    docNumber = int(docNumber.strip())
else:
    match = re.search(' \d', input_string)
    docNumber = 0
    if match != None:
        docNumber = match.group(0)
        input_string = input_string.replace(docNumber, '')
        docNumber = int(docNumber.strip())
input_string = input_string.strip()

original_string = input_string
input_string = input_string.split(' ')
all_possible_strings = get_all_substrings(input_string)
output = [''] * 3
output[0] = get_classified_words(classifier, all_possible_strings, 0).strip()
output[1] = get_classified_words(classifier, all_possible_strings, 1).strip()
output[2] = get_classified_words(classifier, all_possible_strings, 2).strip()

def output_tuning(classifier, original_string, output):
    first_pos = min([original_string.index(output[0]), original_string.index(output[1]), original_string.index(output[2])])
    middle_pos = -1
    last_pos = max([original_string.index(output[0]), original_string.index(output[1]), original_string.index(output[2])])
    first = -1
    middle = -1
    last = -1
    for i in range(0, 3):
        pos = original_string.index(output[i])
        if pos == first_pos:
            first = i
        else:
            if pos == last_pos:
                last = i            
    for i in range(0, 3):
        if i != first and i != last:
            middle = i
            middle_pos = original_string.index(output[middle])
            break
    if output[first] != None and output[first] != '':
        if first_pos > 0:
            output[first] = original_string[:first_pos] + output[first]
            first_pos = 0
    if output[last] != None and output[last] != '':
        if last_pos + len(output[last]) != len(original_string):
            output[last] = output[last] + original_string[last_pos + len(output[last]):]
    if first_pos + len(output[first]) != middle_pos:
        output_first = output[first] + original_string[first_pos + len(output[first]):middle_pos]
        output_first = output_first.strip()
        output_first_prob = classifier.decision_function(np.array([output_first]))[0][first]
        output_middle = original_string[first_pos + len(output[first]):middle_pos] + output[middle]
        output_middle = output_middle.strip()
        output_middle_prob = classifier.decision_function(np.array([output_middle]))[0][middle]
        if (output_first_prob > output_middle_prob):
            output[first] = output_first
        else:
            output[middle] = output_middle
            middle_pos = first_pos + len(output[first]) + 1
    if middle_pos + len(output[middle]) != last_pos:
        output_middle = output[middle] + original_string[middle_pos + len(output[middle]):last_pos]
        output_middle = output_middle.strip()
        output_middle_prob = classifier.decision_function(np.array([output_middle]))[0][middle]
        output_last = original_string[middle_pos + len(output[middle]) + 1:last_pos] + output[last]
        output_last = output_last.strip()
        output_last_prob = classifier.decision_function(np.array([output_last]))[0][last]
        if (output_middle_prob > output_last_prob):
            output[middle] = output_middle
        else:
            output[last] = output_last
    return output

countNone = 0
if output[0] == '':
    countNone += 1
if output[1] == '':
    countNone += 1
if output[2] == '':
    countNone += 1
if countNone < 2:
    output = output_tuning(classifier, original_string, output)
    print('Tác giả:', output[0])
    if docNumber != 0 and output[0] != '':
        print('Tên tài liệu:', output[1], docNumber)
    else:
        print('Tên tài liệu:', output[1])
    print('Nhà xuất bản:', output[2])
else:
    target_names = ['Tác giả', 'Tên tài liệu', 'Nhà xuất bản']
    predicted = classifier.predict(np.array([original_string]))
    if predicted[0] != 1:
        print(target_names[predicted[0]], ': ', original_string, sep='')
    else:
        if docNumber != 0:
            print('Tên tài liệu:', original_string, docNumber)
        else:
            print('Tên tài liệu:', original_string)
if pubYear != 0:
    print('Năm xuất bản:', pubYear)
else:
    print('Năm xuất bản:')

Đầu vào: Chia sẻ tâm hồn và quà tặng cuộc sống Trần Bình
Tác giả: sống Trần Bình
Tên tài liệu: Chia sẻ tâm hồn và quà tặng cuộc
Nhà xuất bản: 
Năm xuất bản:
